In [4]:
import FinanceDataReader as fdr
import time
from collections import deque
import datetime
import pandas as pd
import filter
import atr
import Long_order
import pandas_market_calendars as mcal # 영업일 계산 모듈

# 매게변수 ticker 추가 
def get_balance_coin(balances, ticker):
    bool = True
    while True:
        # for문 추가
        for i in range(1, 11):
            if ticker in balances[i]['currency']:
                balance = balances[i]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                value = (float(avg_buy_price), float(balance))
                bal_loc = int(i)
                bool = False
                break
        if bool == False:
            break
        else:
            pass
        for j in range(1, 11):
            if balances[j]['currency'] == '':
                value = (int(0), int(0))
                bal_loc = int(j)
                break
        break
    return value, bal_loc

# 매게변수 ticker 추가
def get_ticker_ma(volume, df_ticker, balances, ticker, atr_value):

    now_price = df_ticker.iloc[-1]['Close']                     # 코인 현재가
    coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0][0]                         # 매수 평균가
    balance = coin_check[0][1]                             # 코인 보유 개수
    # 코인의 지갑 위치값 추가
    bal_loc = coin_check[1]
    print("지갑 위치 = {}".format(bal_loc))
    # 손절매 가격
    sell_price = avg_price - (5 * atr_value)
    # 만약 현재가격이 평균가 보다 높아지면 손절매의 가격을 높인다.
    if now_price > avg_price:
        sell_price = now_price - (5 * atr_value)
    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        # 살 가격 정하기
        # 총자산의 10퍼
        buy_price = volume / empty_count
        Long_order.long_buy_order(buy_price, balances, bal_loc, ticker, df_ticker)
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)
        if avg_price < sell_price:
            Long_order.long_sell_order(balances, bal_loc, ticker, df_ticker)
        else:
            print(f"코인명: {ticker}, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))
            print('---------------------------------------------')

i = 0
tickers = []
ma10 = []
ma25 = []
ma50 = []
balances = []
# 시작 날짜 입력
day = datetime.datetime(2022,7,7)
# 초기 티커 필터
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
# 초기 지갑 잔고 입력
volume = 10000

balances = ({'currency': 'dolor', 'balance': '{}'.format(volume), 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} #1
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 5
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 9 실제 딕셔너리 주소
, {'currency': '', 'balance': '', 'avg_buy_price': ''})
empty_count = 9
while True:
    assets = 0
    print('---------------------------------------------')
    print(day)
    startday = day - datetime.timedelta(days = 60)
    while True:
        startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
        endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
        count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
        if count == 50:
            break
        startday -= datetime.timedelta(days = 1)

    # S&P500 주가를 넣어야 한다 수정요망
    df = fdr.DataReader(symbol='US500', start = startday_change, end = endday_change)
    
    ma10 = df['Close'].rolling(window=10).mean()
    df.insert(len(df.columns), "MA10", ma10)

    ma25 = df['Close'].rolling(window=25).mean()
    df.insert(len(df.columns), "MA25", ma25)

    ma50 = df['Close'].rolling(window=50).mean()
    df.insert(len(df.columns), "MA50", ma50)

    volume = float(balances[0]['balance'])

    if df.iloc[-1]['MA10'] > df.iloc[-1]['MA25'] and df.iloc[-1]['Close'] > df.iloc[-1]['MA50']:
        for k in range(1,11):
            if balances[k]['currency'] == '':
                empty_count += 1
                start_filter = True
                break
                
        if start_filter == True:
            tickers = filter.filter(day, endday_change)
        start_filter = False

        # 롱 주문
        for j in range(len(tickers)):
            ticker = tickers[j]
            df_ticker = fdr.DataReader(symbol=ticker, start= startday_change, end=endday_change)
            atr_value = atr.ATR(day, endday_change, ticker)
            get_ticker_ma(volume, df_ticker, balances, ticker, atr_value)
            now_price = df_ticker.iloc[-1]['Close']                     # 코인 현재가
            coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
            balance = coin_check[0][1]                             # 코인 보유 개수
            assets += now_price * balance
        empty_count = 0

    elif df.iloc[-1]['MA10'] < df.iloc[-1]['MA25'] or df.iloc[-1]['Close'] < df.iloc[-1]['MA50']:
        # 가지고 있던 코인 팔기
        for j in range(len(tickers)):
            ticker = tickers[j]
            df_ticker = fdr.DataReader(symbol=ticker, start= startday_change, end=endday_change)
            now_price = df_ticker.iloc[-1]['Close']
            print('now_price = ', now_price)
            coin_check = get_balance_coin(balances, ticker)
            avg_price = coin_check[0][0]
            bal_loc = coin_check[1]
            if type(avg_price) == float:
                Long_order.long_sell_order(balances, bal_loc, ticker, df_ticker)
        
        print('기다리는중')

    assets += float(balances[0]['balance'])

    print("총 평가 자산 = {:,.0f}달러".format(assets))
    # 시간에 24시간씩 계속 추가
    day = day + datetime.timedelta(hours = 24)

    if day == datetime.datetime(2022, 9, 1):
        break
    i += 1
    time.sleep(0.2)


---------------------------------------------
2022-07-07 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-08 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-09 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-10 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-11 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-12 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-13 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-14 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-15 00:00:00
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-16 00:00:00
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2022-07-17 00:00:00
기다리는중
총 평가 자산 = 10,000달러
-----

KeyError: 'close'

In [3]:
import login
import pandas as pd
binance = login.login()
markets = binance.fetch_tickers()
markets = pd.DataFrame(markets)
markets

,ETH/BTC,LTC/BTC,BNB/BTC,NEO/BTC,QTUM/ETH,EOS/ETH,SNT/ETH,BNT/ETH,BCC/BTC,GAS/BTC,...,APT/EUR,APT/TRY,APT/BRL,QKC/BUSD,OSMO/BTC,OSMO/USDT,OSMO/BUSD,HFT/BTC,HFT/BUSD,HFT/USDT
symbol,ETH/BTC,LTC/BTC,BNB/BTC,NEO/BTC,QTUM/ETH,EOS/ETH,SNT/ETH,BNT/ETH,BCC/BTC,GAS/BTC,...,APT/EUR,APT/TRY,APT/BRL,QKC/BUSD,OSMO/BTC,OSMO/USDT,OSMO/BUSD,HFT/BTC,HFT/BUSD,HFT/USDT
timestamp,1668614250875,1668614251017,1668614251027,1668614250594,1668614194415,1668614249763,1668614239819,1668614220058,1668504053788,1668614220894,...,1668614249745,1668614250977,1668614247527,1668614249512,1668614249892,1668614248702,1668613979307,1668614250644,1668614250523,1668614250743
datetime,2022-11-16T15:57:30.875Z,2022-11-16T15:57:31.017Z,2022-11-16T15:57:31.027Z,2022-11-16T15:57:30.594Z,2022-11-16T15:56:34.415Z,2022-11-16T15:57:29.763Z,2022-11-16T15:57:19.819Z,2022-11-16T15:57:00.058Z,2022-11-15T09:20:53.788Z,2022-11-16T15:57:00.894Z,...,2022-11-16T15:57:29.745Z,2022-11-16T15:57:30.977Z,2022-11-16T15:57:27.527Z,2022-11-16T15:57:29.512Z,2022-11-16T15:57:29.892Z,2022-11-16T15:57:28.702Z,2022-11-16T15:52:59.307Z,2022-11-16T15:57:30.644Z,2022-11-16T15:57:30.523Z,2022-11-16T15:57:30.743Z
high,0.074857,0.003555,0.016568,0.000409,0.001714,0.000761,0.000018,0.0003,0.0,0.000133,...,4.3392,85.6,23.53,0.008143,0.000071,1.204,1.203,0.000039,0.6629,0.664
low,0.072455,0.003419,0.016,0.000399,0.001659,0.000732,0.000017,0.000293,0.0,0.000122,...,3.9979,79.3,22.11,0.007632,0.000067,1.12,1.121,0.000034,0.5632,0.5627
bid,0.072564,0.003472,0.016424,0.0004,0.001693,0.000758,0.000018,0.000294,0.0,0.000123,...,4.0995,81.5,22.59,0.007625,0.000068,1.121,1.121,0.000035,0.5723,0.573
bidVolume,10.5594,12.409,6.01,155.62,334.5,2509.7,4663.0,1692.6,0.0,12.4,...,249.67,74.87,107.47,4068.0,34.26,450.89,73.68,740.1,489.8,480.3
ask,0.072565,0.003473,0.016425,0.000401,0.001699,0.000761,0.000018,0.000297,0.0,0.000124,...,4.1168,81.6,22.76,0.007668,0.000068,1.124,1.122,0.000035,0.5731,0.5737
askVolume,1.6442,10.7,7.224,292.63,358.1,3067.7,50000.0,283.6,0.0,182.3,...,701.27,46.76,233.76,4068.0,445.43,364.31,612.49,781.9,499.3,477.9
vwap,0.073868,0.003485,0.016392,0.000404,0.001678,0.000744,0.000018,0.000295,0.0,0.000125,...,4.122662,82.208613,22.780508,0.007834,0.000069,1.167353,1.166337,0.000036,0.610663,0.610736


In [3]:

def filter(day, endday_change):
    # Step.1 5달러 이하 종목 삭제
    print('※ 5달러 이하 종목 삭제 중 ※')
    sort_list = []
    df_SP500 = fdr.StockListing('SP500')
    tickers = list(df_SP500['Symbol'])
    tickers_5 = []
    startday = day - datetime.timedelta(days = 1)
    while True:
        startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
        endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
        count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
        if count == 1:
            break
        startday = startday - datetime.timedelta(days = 1)
    
    for i in range(len(tickers)):
        try:
            ticker = tickers[i]
            ticker_now_price = fdr.DataReader(symbol=ticker, start=startday_change, end=endday_change)
            ticker_price = ticker_now_price.iloc[0]['Close']
            if ticker_price > 50:
                tickers_5.append(ticker)
            print(ticker, ': 완료')
        except:
            print('예외발생')
            pass
    
    print('※ 5달러 이하 종목 삭제 완료 ※')
    sort_list = filter_2(tickers_5, day, endday_change)
    # ticker_5에 5달러 이상 종목 저장
    return sort_list


def filter_2(tickers_5, day, endday_change):
    print(len(tickers_5))
    # Step.2 20일 평균 거래 금액 5,000만달러 이하 삭제
    print('※ 평균 거래 금액 5천만달러 이하 삭제 중 ※')
    j = 0
    sort_list = []
    
    volume_total = pd.DataFrame(columns=['ticker', 'volume'])

    
    startday = day - datetime.timedelta(days = 25)
    while True:
        startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
        endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
        count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
        if count == 20:
            break
        startday = startday - datetime.timedelta(days = 1)

    while True:
        # j번째 USDT 종목을 선택한다.
        ticker = tickers_5[j]
        # 티커의 자료를 받는다.
        df = fdr.DataReader(symbol=ticker, start=startday_change, end=endday_change)
        # 이전 ticker의 값 초기화
        dolor_value = []
        dolor_total = 0
        # 하루 거래량 구한다.
        for i in range(len(df)):
            dolor = float(df.iloc[i]['Close']) * float(df.iloc[i]['Volume'])
            dolor_value.append(dolor)
        df.insert(len(df.columns), "dolor", dolor_value)
        # 20일 평균 거래량 구한다.
        for i in range(len(df)):
            dolor_total += df.iloc[i]['dolor']
        dolor_total = int(dolor_total) / 20

        if dolor_total > 500000000:
            volume_total.loc[j] = [ticker, dolor_total]
        j += 1
        if j == len(tickers_5):
            break
    # 인덱스 리셋 (key값 오류뜸)
    volume_total = volume_total.reset_index(drop=True)
    print('※ 평균 거래 금액 5천만달러 이하 삭제 완료 ※')
    sort_list = filter_3(volume_total, day, endday_change)
    return sort_list

def filter_3(volume_total, day, endday_change):
    print(len(volume_total))
    # Step.3 변동률 순위 구하기
    print('※ 변동률 순위 구하는 중 ※')
    # cs를 저장할 데이터 프레임
    j = 0
    USDT_cs = pd.DataFrame(columns=['ticker', 'cs'])
    while True:
        # j번째 USDT 종목을 선택한다.
        ticker = volume_total.iloc[j]['ticker']
        # 티커의 자료를 받는다.
        
        startday = day - datetime.timedelta(days = 250)
        while True:
            startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
            endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
            count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
            if count == 200:
                break
            startday = startday - datetime.timedelta(days = 1)

        df = fdr.DataReader(symbol=ticker, start=startday_change, end=endday_change)
        # cs와 dpc 초기화
        dpc_total = []
        cs_total = []
        for i in range(len(df)):
            # dpc를 추가하는 코드
            if i > 0:
                dpc = (float(df.iloc[i]['Close']) - float(df.iloc[i - 1]['Close']))/float(df.iloc[i-1]['Close'])*100
                dpc_total.append(dpc)
            else:
                dpc = 0
                dpc_total.append(dpc)
        df.insert(len(df.columns), "dpc", dpc_total)
    
        for i in range(len(df)):
            # cs를 추가하는 코드
            if i > 0:
                cs = float(df.iloc[i]['dpc']) + float(cs)
                cs_total.append(cs)
            else:
                cs = 0
                cs_total.append(cs)
        df.insert(len(df.columns), 'cs', cs_total)
    
        # cs값을 절대값으로 구하는 코드
        cs_value = abs(float(df.iloc[len(df)-1]['cs']))
        USDT_cs.loc[j] = [ticker, cs_value]
        j += 1
        if j == len(volume_total):
            break
    USDT_cs_sort = USDT_cs.sort_values('cs', ascending=False)
    USDT_cs_sort = USDT_cs_sort.reset_index(drop=True)
    sort_list = []
    for sort in range(10):
        sort_list.append(USDT_cs_sort.iloc[sort][0])
    print('※ 변동률 순위 구하기 완료 ※')
    return sort_list


In [4]:
import datetime
import FinanceDataReader as fdr
import pandas_market_calendars as mcal
import pandas as pd

day = datetime.datetime(2022,5,1)
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
filter(day, endday_change)

※ 5달러 이하 종목 삭제 중 ※
MMM : 완료
AOS : 완료
ABT : 완료
ABBV : 완료
ABMD : 완료
ACN : 완료
ATVI : 완료
ADM : 완료
ADBE : 완료
ADP : 완료
AAP : 완료
AES : 완료
AFL : 완료
A : 완료
APD : 완료
AKAM : 완료
ALK : 완료
ALB : 완료
ARE : 완료
ALGN : 완료
ALLE : 완료
LNT : 완료
ALL : 완료
GOOGL : 완료
GOOG : 완료
MO : 완료
AMZN : 완료
AMCR : 완료
AMD : 완료
AEE : 완료
AAL : 완료
AEP : 완료
AXP : 완료
AIG : 완료
AMT : 완료
AWK : 완료
AMP : 완료
ABC : 완료
AME : 완료
AMGN : 완료
APH : 완료
ADI : 완료
ANSS : 완료
AON : 완료
APA : 완료
AAPL : 완료
AMAT : 완료
APTV : 완료
ACGL : 완료
ANET : 완료
AJG : 완료
AIZ : 완료
T : 완료
ATO : 완료
ADSK : 완료
AZO : 완료
AVB : 완료
AVY : 완료
BKR : 완료
BALL : 완료
BAC : 완료
BBWI : 완료
BAX : 완료
BDX : 완료
WRB : 완료
HTTP Error 404: Not Found  - symbol "BRKB"not found or invalid periods
예외발생
BBY : 완료
BIO : 완료
TECH : 완료
BIIB : 완료
BLK : 완료
BK : 완료
BA : 완료
BKNG : 완료
BWA : 완료
BXP : 완료
BSX : 완료
BMY : 완료
AVGO : 완료
BR : 완료
BRO : 완료
HTTP Error 404: Not Found  - symbol "BFB"not found or invalid periods
예외발생
CHRW : 완료
CDNS : 완료
CZR : 완료
CPT : 완료
CPB : 완료
COF : 완료
CAH : 완료
KMX : 완료
CCL : 완료
CARR : 완료

['PYPL', 'DVN', 'OXY', 'NFLX', 'MOS', 'NUE', 'COP', 'CVX', 'CHTR', 'MRNA']

In [31]:

day = datetime.datetime(2022,5,1)
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
startday = day - datetime.timedelta(days = 250)
while True:
    startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
    endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
    count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
    if count == 200:
        break
    startday = startday - datetime.timedelta(days = 1)

df = fdr.DataReader(symbol='MMM', start=startday_change, end=endday_change)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-16,203.119995,203.210007,198.910004,199.369995,190.483826,2474100
2021-07-19,197.789993,198.539993,195.110001,197.559998,188.754486,3146300
2021-07-20,198.339996,202.220001,198.070007,200.820007,191.869217,2783300
2021-07-21,201.130005,202.880005,199.960007,200.770004,191.821426,1999100
2021-07-22,201.509995,201.649994,198.679993,199.070007,190.197205,1700500
...,...,...,...,...,...,...
2022-04-25,148.710007,149.029999,145.880005,148.600006,145.618912,3432200
2022-04-26,145.410004,146.979996,142.869995,144.220001,141.326767,4840400
2022-04-27,144.779999,146.080002,143.729996,144.210007,141.316971,3124300


In [43]:
# cs와 dpc 초기화
dpc_total = []
cs_total = []
for i in range(len(df)):
    # dpc를 추가하는 코드
    if i > 0:
        dpc = (float(df.iloc[i]['Close']) - float(df.iloc[i - 1]['Close']))/float(df.iloc[i-1]['Close'])*100
        dpc_total.append(dpc)
    else:
        dpc = 0
        dpc_total.append(dpc)
df.insert(len(df.columns), "dpc", dpc_total)

for i in range(len(df)):
    # cs를 추가하는 코드
    if i > 0:
        cs = float(df.iloc[i]['dpc']) + float(cs)
        cs_total.append(cs)
    else:
        cs = 0
        cs_total.append(cs)
df.insert(len(df.columns), 'cs', cs_total)
df
# # cs값을 절대값으로 구하는 코드
# cs_value = abs(float(df.loc[len(df)-1, 'cs']))
# USDT_cs.loc[j] = [ticker, cs_value]
# j += 1
# if j == len(volume_total):
#     break

ValueError: cannot insert dpc, already exists

In [44]:
df

,Open,High,Low,Close,Adj Close,Volume,dpc,cs
Date,,,,,,,,
2021-07-16,203.119995,203.210007,198.910004,199.369995,190.483826,2474100,0.000000,0.000000
2021-07-19,197.789993,198.539993,195.110001,197.559998,188.754486,3146300,-0.907858,-0.907858
2021-07-20,198.339996,202.220001,198.070007,200.820007,191.869217,2783300,1.650136,0.742278
2021-07-21,201.130005,202.880005,199.960007,200.770004,191.821426,1999100,-0.024899,0.717378
2021-07-22,201.509995,201.649994,198.679993,199.070007,190.197205,1700500,-0.846739,-0.129360
...,...,...,...,...,...,...,...,...
2022-04-25,148.710007,149.029999,145.880005,148.600006,145.618912,3432200,-0.382109,-27.862361
2022-04-26,145.410004,146.979996,142.869995,144.220001,141.326767,4840400,-2.947513,-30.809875
2022-04-27,144.779999,146.080002,143.729996,144.210007,141.316971,3124300,-0.006930,-30.816804


In [17]:
import datetime
import time

day = datetime.datetime(2022,5,1)
day = int(time.mktime(day.timetuple())*1000)
ticker = 'BTC/USDT'
interval = '1h'
ticker_now_price = binance.fetch_ohlcv(symbol = ticker, since = day, limit= 1, timeframe = interval)
ticker_now_price = pd.DataFrame(ticker_now_price, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
ticker_now_price 

,date,open,high,low,close,volume
0,1651330800000,38258.27,38529.98,38156.04,38466.56,2450.1886


In [9]:

day = datetime.datetime(2022,5,1)
startday = day - datetime.timedelta(days = 2)
startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
    
ticker_now_price = fdr.DataReader(symbol='MMM', start=startday_change, end=endday_change)
#ticker_price = ticker_now_price.iloc[0]['Close']
ticker_now_price

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-04-29,147.300003,148.559998,143.869995,144.220001,141.326767,4152300


In [2]:
import FinanceDataReader as fdr
import datetime

df_SP500 = fdr.StockListing('SP500')
tickers = list(df_SP500['Symbol'])
ticker_select = tickers[0]

day = datetime.datetime(2020,1,30)
startday = day - datetime.timedelta(days = 30)
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
    
df = fdr.DataReader(symbol=ticker_select, start=startday_change, end=endday_change)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,175.229996,176.589996,175.070007,176.419998,160.000183,1573500
2020-01-02,177.679993,180.009995,177.139999,180.000000,163.246994,3601700
2020-01-03,177.020004,178.660004,175.630005,178.449997,161.841263,2466900
2020-01-06,177.149994,178.710007,176.350006,178.619995,161.995453,1998000
2020-01-07,178.279999,178.509995,176.820007,177.899994,161.342422,2173000
2020-01-08,178.000000,181.500000,177.649994,180.630005,163.818390,2758300
2020-01-09,181.509995,181.589996,179.759995,181.199997,164.335297,2746300
2020-01-10,181.610001,182.179993,180.139999,180.470001,163.673218,2103800
2020-01-13,180.940002,182.229996,180.360001,180.919998,164.081375,2090800


In [32]:
import pandas_market_calendars as mcal

day = datetime.datetime(2020,1,1)
startday = day - datetime.timedelta(days = 25)
while True:
    startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
    endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
    count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
    if count == 20:
        print(startday)
        break
    startday = startday - datetime.timedelta(days = 1)


2019-12-03 00:00:00


20

In [35]:
import pandas_market_calendars as mcal

day = datetime.datetime(2020,1,30)
startday = day - datetime.timedelta(days = 35)

startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)

early = mcal.get_calendar('NYSE').schedule(start_date='2019-12-03', end_date='2020-01-01')
len(early)

20

In [12]:
type(day.year)

int

In [10]:
date = '201711081750'

np.datetime64( date[:4] +'-'+date[4:6]+'-'+date[6:8]+' ' +date[8:10]+':'+date[10:]  , 'm' )


numpy.datetime64('2017-11-08T17:50')

In [24]:
df.iloc[0]['Close']

176.419998

In [25]:

df_SP500 = fdr.StockListing('SP500')
tickers = list(df_SP500['Symbol'])
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRKB',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BFB',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA'

In [39]:
df = fdr.DataReader(symbol='MMM', start=startday_change, end=endday_change)
type(df)

pandas.core.frame.DataFrame